In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.neural_network import MLPClassifier

import pandas as pd
import numpy as np

features_dataframe = pd.read_csv('/content/drive/MyDrive/Research/Final GAB files/Features9jan21_70.csv')

def conditions(s):
  return s['Sentiment_score_entire_cascade'] - s['Sentiment_score_upto_level_k']

def average_conditions(s):
  return s['Average_sentiment_entire_cascade'] - s['Average_sentiment_upto_k']

def positive_conditions(s):
  return s['Positive_nodes_entire_cascade'] - s['Positive_nodes_upto_k']

def negative_conditions(s):
  return s['Negative_nodes_entire_cascade'] - s['Negative_nodes_upto_k']

def get_followers_after_k(s):
  return s['Followers_entire_cascade'] - s['Followers_upto_k']

def get_followers_after_k(s):
  return s['Total_nodes_entire_cascade'] - s['Total_nodes_upto_k']

def getPolarityChange(s):
  if s['Sentiment_score_upto_level_k']*s['Sentiment_score_after_k'] > 0:
    return 0
  else:
    return 1

def get_polarity_after_k(s):
  if s['Sentiment_score_after_k'] >0:
    return 1
  else:
    return 0

features_dataframe['Sentiment_score_after_k'] = features_dataframe.apply(conditions, axis=1)
features_dataframe['Followers_after_k'] = features_dataframe.apply(get_followers_after_k, axis=1)
features_dataframe['Average_sentiment_after_k'] = features_dataframe.apply(average_conditions, axis=1)
features_dataframe['Positive_nodes_after_k'] = features_dataframe.apply(positive_conditions, axis=1)
features_dataframe['Negative_nodes_after_k'] = features_dataframe.apply(negative_conditions, axis=1)
features_dataframe['Polarity_change_after_k'] = features_dataframe.apply(getPolarityChange, axis=1)
features_dataframe['Polarity_after_k'] = features_dataframe.apply(get_polarity_after_k, axis=1)
features_dataframe['Total_nodes_after_k'] = features_dataframe.apply(get_polarity_after_k, axis=1)

counts = features_dataframe['Polarity_change_after_k'].value_counts().to_dict()
print("Polarity change after k")
print(counts)

counts = features_dataframe['Polarity_after_k'].value_counts().to_dict()
print("Polarity after k")
print(counts)

#X = features_dataframe[['k_levels','Total_nodes_upto_k','Positive_edges_upto_k','Negative_edges_upto_k','Positive_nodes_upto_k','Negative_nodes_upto_k','Sentiment_score_upto_level_k','Average_sentiment_upto_k','Total_users_upto_k']]
#ALL#X = features_dataframe[['Total_nodes_upto_k','Total_users_upto_k','Cascade_influence','followers_influence','border_influence','ineffectivity','Positive_nodes_upto_k','Negative_nodes_upto_k','Sentiment_score_upto_level_k','Average_sentiment_upto_k','Positive_edges_upto_k','Negative_edges_upto_k','Root_followers_upto_k','Total_root_followers']]
#USER#X = features_dataframe[['Total_users_upto_k','Cascade_influence','followers_influence','border_influence','ineffectivity','Root_followers_upto_k','Total_root_followers']]
X = features_dataframe[['Positive_nodes_upto_k','Negative_nodes_upto_k','Sentiment_score_upto_level_k','Average_sentiment_upto_k','Positive_edges_upto_k','Negative_edges_upto_k']]
transformer_object = Normalizer().fit(X)
transformer = transformer_object.transform(X)

Y = features_dataframe['Polarity_change_after_k']
Z = features_dataframe['Polarity_after_k']

Polarity change after k
{0: 7735, 1: 3189}
Polarity after k
{1: 8651, 0: 2273}


In [2]:
#RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(transformer, Y, test_size=0.20, random_state=0)
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
random_forest_change = classifier.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(transformer, Z, test_size=0.20, random_state=0)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
random_forest = classifier.score(X_test, y_test)

In [3]:
#NN Multilayer perceptron

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
X_train, X_test, y_train, y_test = train_test_split(transformer, Y, test_size=0.20, random_state=0)
clf = MLPClassifier(random_state=1, max_iter=100).fit(X_train, y_train)
clf.predict_proba(X_test[:])

clf.predict(X_test[:, :])

multilayer_perceptron_change = clf.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(transformer, Z, test_size=0.20, random_state=0)

clf = MLPClassifier(random_state=1, max_iter=100).fit(X_train, y_train)
clf.predict_proba(X_test[:])

clf.predict(X_test[:, :])

multilayer_perceptron = clf.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [4]:
#Decision Tree Classifier

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(transformer, Y, test_size=0.20, random_state=0)
clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
clf.predict_proba(X_test[:])

clf.predict(X_test[:, :])

decision_tree_change = clf.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(transformer, Z, test_size=0.20, random_state=0)

clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
clf.predict_proba(X_test[:])

clf.predict(X_test[:, :])

decision_tree = clf.score(X_test, y_test)

In [5]:
#GaussianNB

from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(transformer, Y, test_size=0.20, random_state=0)
clf = GaussianNB()
clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
gaussianNB_change = clf.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(transformer, Z, test_size=0.20, random_state=0)

clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
gaussianNB = clf.score(X_test, y_test)

In [6]:
#Quadratic Discriminant Analysis

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
X_train, X_test, y_train, y_test = train_test_split(transformer, Y, test_size=0.20, random_state=0)
clf = QuadraticDiscriminantAnalysis()
clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
quadratic_discriminant_change = clf.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(transformer, Z, test_size=0.20, random_state=0)

clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
quadratic_discriminant = clf.score(X_test, y_test)

In [7]:
#Ada Boost Classifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
X_train, X_test, y_train, y_test = train_test_split(transformer, Y, test_size=0.20, random_state=0)
clf = AdaBoostClassifier(n_estimators=100, random_state=0)

clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
ada_boost_classifier_change = clf.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(transformer, Z, test_size=0.20, random_state=0)

clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
ada_boost_classifier = clf.score(X_test, y_test)

In [8]:
#K nearest neighbor algorithm

from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(transformer, Y, test_size=0.20, random_state=0)
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
k_nearest_neighbor_change = clf.score(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(transformer, Z, test_size=0.20, random_state=0)

clf.fit(X_train, y_train)

clf.predict(X_test[:, :])
k_nearest_neighbor = clf.score(X_test, y_test)

In [9]:
print("Random Forest:         ", round(random_forest_change, 8))
print("Multi Layer Perceptron:", round(multilayer_perceptron_change, 8))
print("Decision Tree:         ", round(decision_tree_change, 8))
print("Gaussian NB:           ", round(gaussianNB_change, 8))
print("Quadratic Discriminant:", round(quadratic_discriminant_change, 8))
print("Ada Boost:             ", round(ada_boost_classifier_change, 8))
#print("K Nearest Neighbors:   ", round(k_nearest_neighbor_change, 8))

Random Forest:          0.76613272
Multi Layer Perceptron: 0.77528604
Decision Tree:          0.68466819
Gaussian NB:            0.73043478
Quadratic Discriminant: 0.75377574
Ada Boost:              0.79588101


In [10]:
print("Random Forest:         ", round(random_forest, 8))
print("Multi Layer Perceptron:", round(multilayer_perceptron, 8))
print("Decision Tree:         ", round(decision_tree, 8))
print("Gaussian NB:           ", round(gaussianNB, 8))
print("Quadratic Discriminant:", round(quadratic_discriminant, 8))
print("Ada Boost:             ", round(ada_boost_classifier, 8))
#print("K Nearest Neighbors:   ", round(k_nearest_neighbor, 8))


Random Forest:          0.76018307
Multi Layer Perceptron: 0.79771167
Decision Tree:          0.6617849
Gaussian NB:            0.75240275
Quadratic Discriminant: 0.7784897
Ada Boost:              0.79542334
